In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Delta").getOrCreate()

data = [(1, "Alice", 29), (2, "Bob", 35)]
columns = ["id", "name", "age"]
df = spark.createDataFrame(data, columns)
delta_path = "/D:/Hexa2024/Delta Tables"
# Write the DataFrame as a Delta table
df.write.format("delta").mode("overwrite").save(delta_path)


In [0]:
df_delta = spark.read.format("delta").load(delta_path)

In [0]:
df_delta.show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|Alice| 29|
|  2|  Bob| 35|
+---+-----+---+



In [0]:
#Converting from Parquet to Delta Table
# #%sql
#CONVERT TO DELTA parquet.`\path\to parquet file on cloud`

In [0]:
''' Creating Delta Table using SQL
CREATE TABLE Delta_Table
USING DELTA AS
SELECT * FROM hexawarede.default.person;'''

' Creating Delta Table using SQL\nCREATE TABLE Delta_Table\nUSING DELTA AS\nSELECT * FROM hexawarede.default.person;'

In [0]:
#Getting delta table into spark dataframe
person_delta = spark.read.table("hexawarede.default.delta_table")

person_delta.show()


+----------------+----------+---------+-----+---------+----------+----------+------+--------------+---------------------+--------------------+--------------------+-------------------+
|BusinessEntityID|PersonType|NameStyle|Title|FirstName|MiddleName|  LastName|Suffix|EmailPromotion|AdditionalContactInfo|        Demographics|             rowguid|       ModifiedDate|
+----------------+----------+---------+-----+---------+----------+----------+------+--------------+---------------------+--------------------+--------------------+-------------------+
|               1|        EM|    false| NULL|      Ken|         J|   Sánchez|  NULL|             0|                 NULL|<IndividualSurvey...|92C4279F-1207-48A...|2009-01-07 00:00:00|
|               2|        EM|    false| NULL|    Terri|       Lee|     Duffy|  NULL|             1|                 NULL|<IndividualSurvey...|D8763459-8AA8-47C...|2008-01-24 00:00:00|
|               3|        EM|    false| NULL|  Roberto|      NULL|Tamburello|  N

In [0]:
#UPSERT operation using MERGE
'''sql 
MERGE INTO target_table AS t
USING source_table AS s
ON t.id = s.id
WHEN MATCHED THEN UPDATE SET t.name = s.name, t.age = s.age
WHEN NOT MATCHED THEN INSERT (id, name, age) VALUES (s.id, s.name, s.age);'''


In [0]:
%sql
OPTIMIZE hexawarede.default.delta_table;

path,metrics
abfss://unity-catalog-storage@dbstorageerk7kj2b3ojsg.dfs.core.windows.net/4038394663932067/__unitystorage/catalogs/9f862d5c-1fd5-4e61-81c2-2596738add14/tables/58f2ca1d-cd31-4109-8991-f98aa237665c,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 1, true, 0, 0, 1727949507165, 1727949509656, 4, 0, null, List(0, 0), 13, 13, 0, 0, null)"


In [0]:

%sql
OPTIMIZE hexawarede.default.delta_table ZORDER BY (BusinessEntityID);

path,metrics
abfss://unity-catalog-storage@dbstorageerk7kj2b3ojsg.dfs.core.windows.net/4038394663932067/__unitystorage/catalogs/9f862d5c-1fd5-4e61-81c2-2596738add14/tables/58f2ca1d-cd31-4109-8991-f98aa237665c,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 2122674), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1727949531867, 1727949535919, 4, 0, null, List(0, 0), 13, 13, 0, 0, null)"


In [0]:
#Time Travel and Vacuum
'''
%sql
SELECT * FROM hexawarede.default.delta_table VERSION AS OF 5
VACUUM hexawarede.default.delta_table'''


'\n%sql\nSELECT * FROM hexawarede.default.delta_table VERSION AS OF 5\nVACUUM hexawarede.default.delta_table'

 **Hands-on Exercise: Using Delta Lake for Data Versioning**

In [0]:
%sql
CREATE TABLE emp_delta
USING DELTA AS
SELECT * FROM hexawarede.default.employee;

num_affected_rows,num_inserted_rows


In [0]:
%sql
MERGE INTO delta_table AS T
USING emp_delta AS U
ON T.BusinessEntityID = U.BusinessEntityID
WHEN MATCHED THEN UPDATE SET T.Title = U.JobTitle
WHEN NOT MATCHED THEN INSERT (BusinessEntityID, NationalIDNumber, LoginID, OrganizationNode, OrganizationLevel, JobTitle, BirthDate, MaritalStatus, Gender, HireDate, SalariedFlag, VacationHours, SickLeaveHours, CurrentFlag, rowguid, ModifiedDate)
  VALUES (U.BusinessEntityID, U.NationalIDNumber, U.LoginID, U.OrganizationNode, U.OrganizationLevel, U.JobTitle, U.BirthDate, U.MaritalStatus, U.Gender, U.HireDate, U.SalariedFlag, U.VacationHours, U.SickLeaveHours, U.CurrentFlag, U.rowguid, U.ModifiedDate);

In [0]:
%sql
SELECT * FROM delta_table VERSION AS OF 2
OPTIMIZE delta_table

In [0]:
%sql
VACUUM emp_delta

path
abfss://unity-catalog-storage@dbstorageerk7kj2b3ojsg.dfs.core.windows.net/4038394663932067/__unitystorage/catalogs/9f862d5c-1fd5-4e61-81c2-2596738add14/tables/d9e75005-62cd-436d-82e1-e9458b48bc9c


**Hands-on Exercise: Building a Reliable Data Lake with Databricks Delta Lake**


In [0]:
data = [(1, "John"), (2, "Alice"), (3, "Bob")]
columns = ["id", "name"]
raw_data = spark.createDataFrame(data, columns)

# Save as Delta table
raw_data.write.format("delta").mode("overwrite").save("/D:/Hexa2024/Delta Tables")

In [0]:
%sql
CREATE TABLE enforced_table (
    id INT,
    name STRING
)
USING DELTA;


In [0]:
updates = spark.read.format("delta").load("/D:/Hexa2024/Delta Tables")

In [0]:
'''
%sql
MERGE INTO enforced_table AS t
USING updates AS u
ON t.id = u.id
WHEN MATCHED THEN 
    UPDATE SET t.name = u.name
WHEN NOT MATCHED THEN 
    INSERT (id, name) 
    VALUES (u.id, u.name);'''


'\n%sql\nMERGE INTO enforced_table AS t\nUSING updates AS u\nON t.id = u.id\nWHEN MATCHED THEN \n    UPDATE SET t.name = u.name\nWHEN NOT MATCHED THEN \n    INSERT (id, name) \n    VALUES (u.id, u.name);'

In [0]:
%sql
OPTIMIZE enforced_table ZORDER BY (id);

path,metrics
abfss://unity-catalog-storage@dbstorageerk7kj2b3ojsg.dfs.core.windows.net/4038394663932067/__unitystorage/catalogs/9f862d5c-1fd5-4e61-81c2-2596738add14/tables/5393de18-e125-43ba-9e7b-4afc7663d746,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(0, 0), 0, List(0, 0), 0, null), null, 0, 0, 0, 0, false, 0, 0, 1727951447688, 1727951450190, 4, 0, null, List(0, 0), 2, 2, 0, 0, null)"


In [0]:
'''%sql
-- Explore the data at version 5
SELECT * FROM enforced_table VERSION AS OF 5;'''

In [0]:
%sql
-- Clean up old files and retain versions for 7 days (168 hours)
VACUUM enforced_table RETAIN 168 HOURS;
